In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/Colab Notebooks/DSA4213
# !ls -la

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/DSA4213
total 340
-rw------- 1 root root 27866 Nov  9 12:02 01_data_loading_exploration.ipynb
-rw------- 1 root root 70110 Nov  9 12:02 02_ner.ipynb
-rw------- 1 root root 26019 Nov  9 15:46 baseline_no_RAG.ipynb
-rw------- 1 root root 78304 Nov  9 15:38 base_RAG.ipynb
-rw------- 1 root root   395 Nov  9 12:29 .env
drwx------ 2 root root  4096 Nov  9 14:53 __pycache__
-rw------- 1 root root  8708 Nov  9 14:32 query_processor.py
-rw------- 1 root root 32273 Nov  9 14:55 query_processors.ipynb
-rw------- 1 root root 96703 Nov  9 12:02 RAG_post_retrieval.ipynb
-rw------- 1 root root  1082 Nov  9 15:05 requirements.txt


In [2]:
!pip install -r requirements.txt

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached lxml-5.3.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.8 kB)
  Using cached sentence_transformers-5.0.0-py3-none-any.whl.metadata (16 kB)
  Using cached qdrant_client-1.15.1-py3-none-any.whl.metadata (11 kB)
  Using cached openai-2.6.0-py3-none-any.whl.metadata (29 kB)
  Using cached langchain-1.0.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached langchain_qdrant-1.1.0-py3-none-any.whl.metadata (2.0 kB)
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_huggingface-1.0.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached langchain_text_splitters-1.0.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
  Using cached accelerate-0.34.2-py3-none-any.whl.metadata (19 kB)
  Using cached datasets-2.21.0-py3-none-any.whl.metadata (21 

In [3]:
!pip install langchain_openai

In [4]:
# Imports and API Key Setup
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Load API keys from your .env file
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LLM_MODEL = "gpt-4o" # Use the same model as your RAG pipeline for a fair comparison

if not OPENAI_API_KEY:
    print("Error: OPENAI_API_KEY not found. Please set it in your .env file.")
else:
    print(f"OpenAI API Key loaded. Using model: {LLM_MODEL}")

import importlib.util, types, pathlib

module_path = pathlib.Path.cwd() /"query_processor.py"
spec = importlib.util.spec_from_file_location("query_processor", str(module_path))
qp_mod = importlib.util.module_from_spec(spec)
spec.loader.exec_module(qp_mod)

QueryProcessor = qp_mod.QueryProcessor
qp = QueryProcessor()
print("Loaded via spec_from_file_location ✓")

print("QueryProcessor OK")


OpenAI API Key loaded. Using model: gpt-4o


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Loaded via spec_from_file_location ✓
QueryProcessor OK


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


In [7]:
# Define the "Benchmark" Chain (No RAG)

# 1. Initialize the LLM (same as your RAG pipeline)
llm = ChatOpenAI(
    model=LLM_MODEL,
    api_key=OPENAI_API_KEY
)

# 2. Define the Prompt Template
# Neutral baseline prompt (used when use_query_processor=False)
BASELINE_PROMPT = """You are a financial analyst.
Answer the user's question using your general knowledge (no external context).
Avoid fabricating specific numbers/dates; hedge if uncertain.

Question:
{query}

Answer:"""

# Intent-aware prompts (used when use_query_processor=True)
NO_RAG_RISK_PROMPT = """You are a financial analyst.
Answer using general knowledge (no external context). Focus on risk factors (Item 1A), cybersecurity, litigation, regulatory, macro/FX, concentration.
Avoid fabricating numbers/dates; hedge if uncertain.

Question:
{query}

Answer:"""

NO_RAG_PERFORMANCE_PROMPT = """You are a financial analyst.
Answer using general knowledge (no external context). Focus on performance: revenue, margins, growth drivers, guidance, and segment mix.
Avoid fabricating numbers/dates; hedge if uncertain.

Question:
{query}

Answer:"""

NO_RAG_STRATEGY_PROMPT = """You are a financial analyst.
Answer using general knowledge (no external context). Focus on strategy: expansion plans, product/segment bets, acquisitions/partnerships, capex, restructuring.
Avoid fabricating numbers/dates; hedge if uncertain.

Question:
{query}

Answer:"""

def _pick_no_rag_prompt(label: str) -> str:
    if label == "risk": return NO_RAG_RISK_PROMPT
    if label == "performance": return NO_RAG_PERFORMANCE_PROMPT
    return NO_RAG_STRATEGY_PROMPT


# 3. Define a simple string output parser
output_parser = StrOutputParser()

# 4. Create the "benchmark" chain
prompt = ChatPromptTemplate.from_template(BASELINE_PROMPT)
# This is the "base benchmark" chain: Prompt -> LLM -> Output
benchmark_chain = prompt | llm | output_parser

print("Benchmark (non-RAG) chain created successfully.")

Benchmark (non-RAG) chain created successfully.


In [8]:
# Run Queries and Get Answers
# TO BE CHANGED AFTER EVALUATION QA ARE GENERATED
# Use the same questions from your 'base_RAG.ipynb' for a fair comparison
questions = [
    "What are the main risk factors mentioned by tech companies?",
    "What risks did Apple disclose in their latest 10-Q?",
    "Compare the revenue trends of NVIDIA and AMD",
    "What was Tesla's R&D spending in the latest quarter?"
]

print(f"--- STARTING BENCHMARK QUERIES (NO RAG, WITH QUERY PROCESSING) ---")

for q in questions:
    # 1) Process the query (intent, filters, paraphrases, keywords)
    qobj = qp.process(q)
    label = qobj["label"]
    conf  = qobj["confidence"]
    filt  = qobj["filters"]

    # 2) Enrich the input slightly for the LLM (non-binding hints)
    p0 = (qobj["paraphrases"][0] if qobj["paraphrases"] else "")
    kw = " ".join(qobj["keywords"][:6])
    enriched = f"{q}\n\n(Paraphrase: {p0})\n(Keywords: {kw})" if (p0 or kw) else q

    # 3) Choose an intent-aware prompt
    tmpl_str = _pick_no_rag_prompt(label)
    prompt = ChatPromptTemplate.from_template(tmpl_str)

    # 4) Build the chain for THIS question and invoke
    chain = prompt | llm | output_parser
    answer = chain.invoke({"query": enriched})

    # 5) Pretty print
    print(f"\n{'='*70}")
    print(f"QUERY: {q}")
    print(f"Intent: {label}  (conf={conf:.3f})  Filters: {filt}")
    print(f"{'-'*70}")
    print(f"ANSWER (No-RAG + QP):\n{answer}")
    print(f"{'='*70}")

print("\n--- BENCHMARK QUERIES COMPLETE ---")

--- STARTING BENCHMARK QUERIES (NO RAG, WITH QUERY PROCESSING) ---

QUERY: What are the main risk factors mentioned by tech companies?
Intent: risk  (conf=0.579)  Filters: {}
----------------------------------------------------------------------
ANSWER (No-RAG + QP):
Tech companies typically highlight several key risk factors in their disclosures:

1. **Cybersecurity Risks**: The potential for data breaches and cyberattacks is a significant concern. Companies must protect sensitive information and maintain robust security measures to prevent unauthorized access and data loss.

2. **Litigation Risks**: Tech companies face the risk of lawsuits, often related to intellectual property disputes, antitrust issues, or consumer protection cases. Legal challenges can be costly and affect reputation.

3. **Regulatory Risks**: Changes in laws and regulations, particularly concerning data privacy, antitrust, and industry-specific standards, can significantly impact operations. Compliance with diff